In [1]:
%matplotlib widget
import os
import numpy as np
import pandas as pd
import textwrap
from astropy.io import fits
import scipy.constants as const
import single_lc_funcs as fn
import matplotlib.pyplot as plt

In [2]:
tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 13,
    "font.size": 12,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 13,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12
}

plt.rcParams.update(tex_fonts)

In [3]:
#Load in occurence rate tables
fgk = pd.read_csv(os.path.join('Occurrence_rates','norm_rates_FGK.csv'))
tables = {'F': fgk, 'G':fgk, 'K':fgk}

In [4]:
#Filter target list to include only single sector targets
target_list_path = os.path.join('target-lists', 'year1targets.csv')
target_list = pd.read_csv(target_list_path, index_col=None)
target_list['N_sec'] = target_list.drop('OBJECT', axis=1).sum(axis=1)
mask = target_list['N_sec'] == 1
target_list = target_list[mask]
del target_list['N_sec']

In [5]:
#Pick a target
lim = target_list.shape[0]
tic, secs = fn.tic_choose(target_list, np.random.randint(low=0, high=lim))
Paths = fn.spoc_lc_path(tic, secs)

In [6]:
#Generate transit signals and calculate signal to noise
obj = fn.signals(path=Paths[0], number=10, rate_tables=tables, N_b=10, N_phase=10)
obj.get_results()
log = obj.to_pandas()
print(log)

           TIC_ID        RA       DEC  TESSMAG  ST_T_EFF TYPE  PL_PERIOD  \
0    TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  29.347858   
1    TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  29.347858   
2    TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  29.347858   
3    TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  29.347858   
4    TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  29.347858   
..            ...       ...       ...      ...       ...  ...        ...   
995  TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  79.005806   
996  TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  79.005806   
997  TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  79.005806   
998  TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  79.005806   
999  TIC 24208688  14.83155 -9.124406  12.2078    5951.0    G  79.005806   

     PL_RADIUS   P_TRANS     P_OBS    IMPACT    OFFSET       S/N  
0     0.567964  0.01

In [7]:
#Plot signal to noise as a function of period and radius
S_N = log['S/N']
rad = log['PL_RADIUS']
period = log['PL_PERIOD']
fig, ax = plt.subplots(1,2, figsize=[12.8, 4.8], num=1)
ax[0].scatter(x=rad, y=S_N)
ax[0].set_xlabel('Radius (Earth Radii)')
ax[0].set_ylabel('Signal to Noise ratio')
ax[1].scatter(x=period, y =S_N)
ax[1].set_xlabel('Orbital Period (days)')
ax[1].set_ylabel('Signal to noise ratio')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
#Plot histogram of phases
fig, ax = plt.subplots(1,2, figsize=(12.8, 4.8), num=2)
phase_off = np.ndarray.flatten(obj.ph_offset)
T0 = np.ndarray.flatten(obj.T_0)
ax[0].hist(x=phase_off, bins=20)
ax[0].set_xlabel('Phase offset')
ax[1].hist(x=T0, bins=20)
ax[1].set_xlabel('$T_0$')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
N_pl = len(obj.pl_period)
fig, ax = plt.subplots(1, int(N_pl), figsize=[6.4*N_pl, 4.8], num=3)
for i in range(N_pl):
    for j in range(len(obj.b[i])):
        b_str = 'b=' + str(round(obj.b[i,j], 3))
        ax[i].scatter(x=obj.ph_offset[i,j], y=obj.SN[i,j], label=b_str)
        ax[i].set_xlabel('Phase offset')
        ax[i].set_ylabel('Signal to noise ratio')
        title_str = ('Radius ='+str(round(obj.pl_rad[i], 2))+' Period='+str(round(obj.pl_period[i],2)))
        ax[i].set_title(title_str)
        ax[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
#Plot impact parameter vs S/N averaged across all phases
N_pl = len(obj.pl_period)
fig, ax = plt.subplots(1, int(N_pl), figsize=[6.4*N_pl, 4.8], num=4)
for i in range(N_pl):
    S_N = np.empty_like(obj.b[i])
    for j in range(len(obj.b[i])):
        S_N[j] = np.mean(obj.SN[i,j])
    ax[i].scatter(x=obj.b[i], y=S_N)
    ax[i].set_xlabel('Impact parameter')
    ax[i].set_ylabel('Mean S/N')
    title_str = ('Radius ='+str(round(obj.pl_rad[i], 2))+' Period='+str(round(obj.pl_period[i],2)))
    ax[i].set_title(title_str)
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …